In [1]:
import pandas as pd
import re
from collections import Counter
import random
from profanity_filter import ProfanityFilter
from obscene_words_filter.default import get_default_filter
import obscene_words_filter
filter_obscene_regex = get_default_filter()

In [89]:
# df = pd.read_csv("FOR_HYBRID4_to_obscene.csv")
# df = pd.read_csv("../data_text/stories_3mln_new.csv")
# df = pd.read_csv("../data_text/HYBRID_Steps123456.csv")

df_inapp = pd.read_csv("Inappropriate_09_top_vs_one_with_multi.csv")

In [90]:
len(df_inapp)

124597

In [56]:
collected_data = []
counter = 0
total = 300000
with open("C:\\input\\toxic_and_unsafe\\AGGREGATE_RAW_AND_PROCESSED_UNSAFE\\arhivach_data\\all_dvach.txt", encoding = 'utf-8') as f:
    for l in f.readlines():
        collected_data.append(l.strip())
        counter += 1
        if counter == total: break
    

In [57]:
df = pd.DataFrame({'text':collected_data})

In [58]:
df['len'] = df['text'].apply(lambda x: len(x.split()))

In [59]:
df = df[(df['len'] > 5) & (df['len'] < 100)]

In [60]:
df.head()

,text,len
3,МАМА МАМА Я В ИСТОРИИ)))И ТУТ КАПЧИ НЕТУ))),8
5,"Всем привет, с вами Игорь Линк",6
7,"Блядь, покемоны все? Что с серваками?",6
9,> pokpok pok pokвся сутьсажа скрыл,6
10,"Наконец-то вы съехали. Рад за вас, ребят!",7


# Прямой подхол

In [61]:
exclude_list = ['какая','сила','дурак','хрен','дураки','мужу','муж','сосать','сосала',
                'минут','минута','сил','стерва','блин','муд','дот','малафьи','гад','соси','сосать']

In [62]:
mat_initial_form = []
with open("C:\\Users\\N.Babakov\\python_notebooks\\toloka\\data_text\\MAT_FINAL_with_unigram_inflections.txt", 'r', encoding = 'utf-8') as f:
    for m in f.readlines():
        m_cl = re.sub('\n','',m)
        mat_initial_form.append(m_cl)

In [63]:
unigram_mat_set = set()
ngram_mat_set = set()
for mat in mat_initial_form:
    if len(mat.split()) == 1:
        if mat not in exclude_list:
            unigram_mat_set.add(mat)
    else:
        ngram_mat_set.add(mat)

# Регулярки

In [64]:
t = "Да ебнутая, бля ебать уеба это просто ебнутая какой-то!"
for match in filter_obscene_regex.find_bad_word_matches_without_good_words(t): 
    start, end = match.span()
    bad_word = t[start: end]
    print(bad_word)

бля
ебать


# profanity_filter

In [65]:
# !pip install spacy --upgrade
# !python -m spacy download ru_core_news_lg

# Задаем функцию

In [66]:
tsttxt = 'и да, в принципе у нас против Зенита никогда ничего не имели, это москвичи всех бля.'


In [67]:
def is_any_obscene(txt):

    txt = txt.lower()
    
#     for match in filter_obscene_regex.find_bad_word_matches_without_good_words(txt): 
#         start, end = match.span()
#         bad_word = txt[start: end]
#         return bad_word + "_filt"
    
    txt = re.sub('[^a-zA-Zа-яА-Я]', ' ', txt)
    txt = re.sub(' +', ' ', txt)
    txt = txt.split()
    
    any_obscene_unigram = any(word in unigram_mat_set for word in txt)
    if any_obscene_unigram:
        for word in txt:
            if word in unigram_mat_set:
                return word
    
    txt_join = ' '.join(txt)
    any_obscene_ngram = any(ngram in txt_join for ngram in ngram_mat_set)
    if any_obscene_ngram:
        for ngram in ngram_mat_set:
            if ngram in txt_join:
                return ngram
    
    return 'clean'
    
is_any_obscene(tsttxt)

'бля'

# Отфильтровать новый

In [68]:
from tqdm import tqdm

In [69]:
tqdm.pandas()

In [48]:
df_sm = df[:1000]

In [49]:
df_sm['is_obscene'] = df_sm['text'].progress_apply(is_any_obscene)

100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 12409.92it/s]
<ipython-input-49-42e61f5f526c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sm['is_obscene'] = df_sm['text'].progress_apply(is_any_obscene)


In [70]:
df['is_obscene'] = df['text'].progress_apply(is_any_obscene)

100%|███████████████████████████████████████████████████████████████████████| 172152/172152 [00:13<00:00, 13143.34it/s]


In [71]:
len(df[df['is_obscene'] != 'clean'])

52421

In [19]:
Counter(df['is_obscene']).most_common(100)

[('clean', 123666),
 ('кун', 58),
 ('хули', 30),
 ('говна', 22),
 ('сиськи', 20),
 ('нахуя', 18),
 ('мини', 17),
 ('говном', 16),
 ('доту', 15),
 ('обосрался', 13),
 ('говне', 10),
 ('шлюх', 9),
 ('шлюхи', 9),
 ('куны', 9),
 ('жопе', 9),
 ('хуйни', 8),
 ('хуйню', 8),
 ('пиндосы', 7),
 ('дрочу', 7),
 ('дрочил', 7),
 ('дрочат', 7),
 ('жопы', 7),
 ('идиоты', 7),
 ('блять', 7),
 ('охуенная', 6),
 ('дебилы', 6),
 ('охуели', 6),
 ('херню', 6),
 ('ебли', 6),
 ('жопой', 6),
 ('лохов', 6),
 ('ссыт', 6),
 ('ебашил', 5),
 ('дрочки', 5),
 ('съебал', 5),
 ('заебали', 5),
 ('заебало', 5),
 ('подрочил', 5),
 ('охуенный', 5),
 ('ануса', 5),
 ('наебали', 5),
 ('дрочишь', 5),
 ('насрать', 5),
 ('похуй', 5),
 ('охуенное', 4),
 ('проебался', 4),
 ('сисек', 4),
 ('пизд', 4),
 ('срачи', 4),
 ('педики', 4),
 ('обоссал', 4),
 ('нахрена', 4),
 ('еблю', 4),
 ('пиздец', 4),
 ('ебала', 3),
 ('пиндосов', 3),
 ('доте', 3),
 ('шлюхами', 3),
 ('доты', 3),
 ('лохи', 3),
 ('охуительные', 3),
 ('анусы', 3),
 ('хреном', 

In [72]:
df_cleaned = df[df['is_obscene']=='clean']

In [73]:
len(df_cleaned), len(df)

(119731, 172152)

<ipython-input-77-e9cd728004e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['isdup'] = df_cleaned['text'].isin(df_inapp['text'])


In [82]:
import re
def depersonalize(text):
    text = str(text)
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text= re.sub(url_regex, "url", text)
    text = re.sub("id[\d]*",'',text)
    text = re.sub("> ?",'',text)
    text = re.sub("@[\w]*",'',text)
#     text = re.sub("\+[\d]*", "", text)
    text = re.sub("[\d]+", "NUMBER", text)
    text = re.sub(">>", "", text)
#     text = re.sub("[\d]{3,100}", "", text)
    text = re.sub("[\t|\n|\r]", "", text)
#     text = re.sub("[(OP)|url|nickname|phone_number]", "", text)
    return text

In [83]:
df_cleaned['text'] = df_cleaned['text'].apply(depersonalize)

<ipython-input-83-a8c3f4665281>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['text'] = df_cleaned['text'].apply(depersonalize)


In [84]:
df_cleaned['isdup'] = df_cleaned['text'].isin(df_inapp['text'])

<ipython-input-84-e9cd728004e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['isdup'] = df_cleaned['text'].isin(df_inapp['text'])


In [85]:
Counter(df_cleaned['isdup'])

Counter({False: 119524, True: 207})

In [86]:
df_cleaned = df_cleaned[df_cleaned['isdup'] == False]

In [87]:
df_cleaned =df_cleaned[['text']]

In [88]:
df_cleaned.to_csv("for_final_test_no_obscene.csv", index = None)

In [23]:
# df_cleaned.to_csv("FOR_HYBRID4_NO_obscene.csv", index = None)
# df_cleaned.to_csv("FOR_HYBRID5_NO_obscene.csv", index = None)
# df_cleaned.to_csv("../data_text/HFinal_no_obscene_from_dict_proc1.csv.csv", index = None)

In [25]:
# df_cleaned_unsafe = df_cleaned[['text','unsafe']]
# df_cleaned_unsafe.to_csv("../data_text/HFinal_UNSAFE_ONLY_no_obscene_from_dict_proc1.csv", index = None)

# Убираем токсичность

In [86]:
df_autotoxic = pd.read_csv("../data_text/HFinal_no_obs_toxAuto_st2.csv")

In [87]:
df_autotoxic.head()

,text,unsafe,crime_real,crime_web,drugs,gambling,pornography,prostitution,slavery,suicide,...,politics,racism,religion,sex_minorities,sexism,social,human_labeled,is_obscene,abstr,toxic_auto
0,Начальники ДК и нынешнего места работы?,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0,clean,0,0.093190
1,"Читаю. Драки, грабежи, ДТП. Изредка убийства, ...",1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,clean,0,0.178655
2,"Допустимо, если информация не нарушает правил ...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,clean,0,0.016997
3,В больницу при приеме на работу тоже мед комис...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,clean,0,0.003748
4,а зачем ты на проураинские митинги выходил? во...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,clean,0,0.399137


In [88]:
len(df_autotoxic)

163350

In [57]:
import random

In [72]:
random.sample(list(df_autotoxic[(df_autotoxic['toxic_auto']>0.5)&(df_autotoxic['toxic_auto']<0.75)]['text']),10)

['Эти слова гарантируют отсутствие финансирования? Совсем мозги отсохли от двачевания?',
 'я живу в Башкортостане и тут есть марийцы, татары, русские, а башкиров мало. Наверное, выглядят как люди.',
 'Но он не содержит информационной нагрузки о том, говорить или нет. Наркоман, блеадь',
 'Почему не растет борода на щеках?',
 'Куришь сигареты всегда одна, может стреляешь у наркоманов, которые могут и намешать с торчевом?',
 'не отделять логику от интерфейса в Бог мой, скажи, что ты никогда не будешь работать программистом.',
 'Как самостоятельно, быстро и эффективно сломать себе палец?',
 'А с этого места поподробнее, с меня как всегда.теперь это наебизнеса тред',
 'Абу бог наш, а мы свидетели его.',
 'Потому что ты тут один И ты должен проснуться']

In [89]:
df_autotoxic_no_tox = df_autotoxic[df_autotoxic['toxic_auto']<0.75]

In [90]:
len(df_autotoxic_no_tox)

163332

In [91]:
clmn = list(df_autotoxic_no_tox)

In [92]:
clmn

['text',
 'unsafe',
 'crime_real',
 'crime_web',
 'drugs',
 'gambling',
 'pornography',
 'prostitution',
 'slavery',
 'suicide',
 'terrorism',
 'weapons',
 'body_shaming',
 'halth_shaming',
 'politics',
 'racism',
 'religion',
 'sex_minorities',
 'sexism',
 'social',
 'human_labeled',
 'is_obscene',
 'abstr',
 'toxic_auto']

In [93]:
clmn_nec = clmn[:-3] + ['toxic_auto']

In [95]:
len(clmn_nec)

22

In [74]:
df_autotoxic_no_tox.columns

Index(['text', 'unsafe', 'crime_real', 'crime_web', 'drugs', 'gambling',
       'pornography', 'prostitution', 'slavery', 'suicide', 'terrorism',
       'weapons', 'body_shaming', 'halth_shaming', 'politics', 'racism',
       'religion', 'sex_minorities', 'sexism', 'social', 'human_labeled',
       'is_obscene', 'abstr', 'toxic_auto'],
      dtype='object')

In [96]:
df_autotoxic_no_tox = df_autotoxic_no_tox[clmn_nec]

In [97]:
len(df_autotoxic_no_tox)

163332

In [99]:
def round_val(val):
    return round(val,2)

In [100]:
df_autotoxic_no_tox['toxic_auto'] = df_autotoxic_no_tox['toxic_auto'].apply(round_val)

In [101]:
df_autotoxic_no_tox.to_csv("../data_text/HFinal_no_obs_no_tox075_st3.csv", index = None)

# Отфильтровать существующий гибрид, посомтреть на кол-во

In [41]:
def is_any_obscene_v1(txt):

    txt = txt.lower()    
    txt = re.sub('[^a-zA-Zа-яА-Я]', ' ', txt)
    txt = re.sub(' +', ' ', txt)
    txt = txt.split()
    
    any_obscene_unigram = any(word in unigram_mat_set for word in txt)
    if any_obscene_unigram:
        for word in txt:
            if word in unigram_mat_set:
                return word
    
    txt_join = ' '.join(txt)
    any_obscene_ngram = any(ngram in txt_join for ngram in ngram_mat_set)
    if any_obscene_ngram:
        for ngram in ngram_mat_set:
            if ngram in txt_join:
                return ngram
    
    return 'clean'

In [42]:
df_hybrid123 = pd.read_csv("../data_text/HYBRID_ST123_rec_dp4_human_back.csv")

In [43]:
df_hybrid123['is_obscene'] = df_hybrid123['text'].progress_apply(is_any_obscene_v1)

100%|█████████████████████████████████████████████████████████████████████████| 88940/88940 [00:08<00:00, 10157.36it/s]


In [48]:
Counter(df_hybrid123['is_obscene']).most_common(100)

[('clean', 87857),
 ('кун', 88),
 ('хули', 41),
 ('говна', 31),
 ('сиськи', 26),
 ('нахуя', 21),
 ('мини', 20),
 ('доту', 19),
 ('говном', 18),
 ('обосрался', 15),
 ('говне', 15),
 ('куны', 14),
 ('шлюхи', 11),
 ('шлюх', 10),
 ('хуйни', 10),
 ('жопе', 10),
 ('жопы', 9),
 ('дебилы', 9),
 ('хуйню', 9),
 ('охуенная', 8),
 ('лохов', 8),
 ('жопой', 7),
 ('идиоты', 7),
 ('заебало', 7),
 ('ануса', 7),
 ('дрочу', 7),
 ('дрочил', 7),
 ('ебли', 7),
 ('пиндосы', 7),
 ('заебали', 6),
 ('охуели', 6),
 ('охуительные', 6),
 ('дерьма', 6),
 ('дрочат', 6),
 ('охуенный', 6),
 ('ссыт', 6),
 ('дрочки', 5),
 ('доте', 5),
 ('дрочишь', 5),
 ('обосрались', 5),
 ('наебали', 5),
 ('ебашил', 5),
 ('съебал', 5),
 ('подрочил', 5),
 ('обоссал', 5),
 ('срачи', 4),
 ('охуеешь', 4),
 ('сисек', 4),
 ('пиндосам', 4),
 ('педики', 4),
 ('доты', 4),
 ('кунов', 4),
 ('лохи', 4),
 ('проебался', 4),
 ('нахрена', 4),
 ('еблю', 4),
 ('охуенное', 4),
 ('какашки', 4),
 ('наеб', 3),
 ('дебилов', 3),
 ('ебанутые', 3),
 ('ебучих', 3

In [49]:
df_hybrid123_cleaned = df_hybrid123[df_hybrid123['is_obscene']=='clean']

In [50]:
len(df_hybrid123_cleaned), len(df_hybrid123)

(87857, 88940)

In [54]:
len(df_hybrid123_cleaned[df_hybrid123_cleaned['unsafe']>0.75])

19502

In [55]:
len(df_hybrid123[df_hybrid123['unsafe']>0.75])

20333